# 🍎 파이썬 머신러닝 완벽 가이드 혼공

### 2019.04.01 ~ 2019.04.11 교재 03-05장

03. 평가 
04. 분류 
05. 회귀

### 2020-04-01-Wed

진도: 03. 평가 ~ (147쪽 ~) 

## 3장 평가 

머신러닝 과정: 데이터 가공/변환 👉 모델 학습/예측 👉 평가 

여러가지 방법으로 예측 성능 평가하나, 모델이 분류냐 회귀냐에 따라 다름. 회귀의 경우, 실제 값과 예측값의 오차 평균값에 기반하고, 분류의 경우, 이렇게 접근하면 잘못된 평가결과에 빠질 수 있음. 따라서 이 장에서는 분류에 사용되는 성능평가 지표에 대해서 알아볼 것. 특히 0과 1로 결정값이 한정되는 이진 분류의 성능 평가 지표에 대해 집중적으로 설명할 것. 

### 분류의 성능 평가 지표
- 정확도 
- 오차행렬 
- 정밀도 
- 재현율
- F1 스코어 
- ROC AUC 

분류는 이진 분류(긍/부정과 같이 2개의 결괏값만 가지는 것)와 멀티 분류(여러 개의 결정 클래스 값을 가지는 것)로 나눌 수 있음. 왜 이 지표가 이진 분류에서 모두 중요한 지 설명할 것.

### 01. 정확도

- 정확도: 실제 데이터에서 예측 데이터가 얼마나 같은지 판단하는 지표 
- 정확도 = 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수 
- 이진 분류의 경우 데이터의 구성에 따라 ML모델 성능을 왜곡할 수 있음. 

In [3]:
import numpy as np
from sklearn.base import BaseEstimator

# 이 클라스는 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측
class MyDummyClassifier(BaseEstimator):
    # fit( ) 메소드는 아무것도 학습하지 않음. 
    def fit(self , X , y=None):
            pass
    
    # predict( ) 메소드는 단순히 Sex feature가 1 이면 0 , 그렇지 않으면 1 로 예측함. 
    def predict(self, X):
        pred = np.zeros( ( X.shape[0] , 1))
        for i in range (X.shape[0]) :
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else :
                pred[i] = 1
        
        return pred

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습데이터/테스트 데이터 분할. 
titanic_df = pd.read_csv('titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)  # 이전에 전처리 만들어둔 함수
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, \
                                                  test_size=0.2, random_state=0)

# 위에서 생성한 Dummy Classifier를 이용하여 학습/예측/평가 수행. 
myclf = MyDummyClassifier()
myclf.fit(X_train ,y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test , mypredictions)))

Dummy Classifier의 정확도는: 0.7877


이렇게 단순한 알고리즘을 예측해도 데이터 구성에 따라 정확도 결과가 높은 수치로 나올 수 있음. 특히나 정확도는 불균형한 레이블 값 분포에서 ML 모델의 성능을 판단할 때, 적합한 평가 지표가 아님. 

MNIST 데이터 세트는 원래 멀티 레이블 분류나, 예시를 위해 7은 True, 나머지 값은 모두 False로 변환해 불균형한 바이너리 레이블로 분류해봐서 예시를 들 것.

In [9]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self,X,y):
        pass
    
    # 입력값으로 들어오는 X 데이터 셋의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self,X):
        return np.zeros( (len(X),1) , dtype=bool)

# 사이킷런의 내장 데이터 셋인 load_digits( )를 이용하여 MNIST 데이터 로딩
digits = load_digits()

# digits번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 False이고 0으로 변환. 
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split( digits.data, y, random_state=11)